# Técnicas de Prendizaje Automático. Aprendizaje por Refuerzo

## Práctica 1: Librería [Gymnasium](https://gymnasium.farama.org/index.html)


En esta práctica se utilizará [Gymnasium](https://gymnasium.farama.org/index.html) para crear un agente que jugará [Blackjack](https://es.wikipedia.org/wiki/Blackjack). El agente será muy simple, seleccionando las jugadas de forma aleatoria ya que el objetivo es conocer los elementos básicos de Gymnasium.

**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.
<br></br>
- incluya el código requerido entre los apartados:
   - \#\<code>
   - \#\</code>

### Dependencias y utilidades
Ejecute las dos casillas siguientes para instalar algunas dependencias y definir funciones necesarias.


In [49]:
%%capture
!pip install -q swig
!pip install gymnasium[all]

!pip install --upgrade ipykernel

In [50]:
##@title Utilidades
##@markdown Ejecute esta casilla para definir algunas funciones necesarias en la actividad.

import gymnasium as gym
import plotly.express as px
import numpy as np
#from google.colab import output
import time


def visualize_trace(x, y):
  """
  visualiza el promedio de episodios ganados
  """

  ne = np.ones(x.shape[0]) #
  ne = np.cumsum(ne)

  yt = y == 1 # 1 retorno al ganar
  yt = np.cumsum(yt) / ne

  fig = px.line(x=x, y=yt, labels={'x':'Episodio', 'y':'Ganados'})


  fig.update_layout(height=300,
                    width=1200,
                    yaxis_range=[0,1],
                    showlegend=False
                    )
  return fig

### Creación de un entorno

**a)** En Gymnasium, el modelado de un problema mediante aprendizaje por refuerzo gira en torno a los conceptos de **entorno** y **agente** como muestra la figura.

<div>
<img src="https://gymnasium.farama.org/_images/AE_loop.png" width="200"/>
</div>

Gymnasium implementa varios entornos que representan problemas clásicos. Además, permite implementar nuevos entornos para modelar otros problemas. Los entornos contienen cuatro funcionalidades principales `make`, `reset`, `step` y `render`.

En la siguiente casilla, incluya el código necesario para:

**a.1)** Crear e iniciar una instancia del entorno [Blackjack-v1](https://gymnasium.farama.org/environments/toy_text/blackjack/) implementado en Gymnasium. La variable con este entorno debe llamarse `env`.

**a.2)** Imprimir el tamaño del espacio de acciones y la configuración de un estado. El código necesario ya se provee.

**a.3)** Ejecutar un paso de tiempo en el entorno mediante la función `step`. Capture todos los valores que devuelve esta función en las variables `obs_current`, `reward`, `terminated` `truncated` e `info`. Como no se ha implementado el agente, se utilizará la función `sample` del `action_espace` del entorno.

**a.4)** Imprimir el estado actual y la recompensa obtenida. El código necesario ya se provee.

**Sugerencia**: Puede consultar la documentación de Gymnasium para más detalles sobre los [aspectos básicos](https://gymnasium.farama.org/content/basic_usage/) y el entorno [Blackjack-v1](https://gymnasium.farama.org/environments/toy_text/blackjack/).

In [51]:
# a.1 crear entorno
env = gym.make('Blackjack-v1')
obs_past = env.reset()

# a.2
print(f'Espacio de Acciones: {env.action_space}')
print(f'Espacio de Observaciones (estados): {env.observation_space}')

# a.3
action = env.action_space.sample()
obs_current, reward, terminated, truncated, _ = env.step(action)

# a.4
print(f'Estado actual: Puntos={obs_current[0]}  Carta crupier={obs_current[1]}  Aces={obs_current[2]}')

Espacio de Acciones: Discrete(2)
Espacio de Observaciones (estados): Tuple(Discrete(32), Discrete(11), Discrete(2))
Estado actual: Puntos=13  Carta crupier=2  Aces=0


### Creación de un agente

**b)** El otro concepto fundamental en Gymnasium es el de agente. Una forma de implementarlo es mediante una clase que contenga la información necesaria para el aprendizaje y las funciones para indicar que acción va a realizar, y actualizar su conocimiento. En el ejemplo se llaman `get_action` y `update` respectivamente.

En este caso, la única información relevante para el agente es el tamaño del espacio de acciones `action_space_size`, ya que estas se enumeran desde el 0 de forma consecutiva `action_space_size`-1.

En la siguiente casilla, incluya el código necesario para:

**b.1)** Devolver una acción al llamar a la función `get_action`. Esta será un número aleatorio entre 0 y `action_space_size`-1 ambos inclusive.

**b.2)** Crear una instancia del agente, en una variable llamada `agent`.

**Sugerencia**: Investigue sobre los parámetros que reciben cada una de estas funciones.

In [52]:
# b.1

import random
class BlackjackAgentRandom:

    def __init__(
        self,
        action_space_size
    ):
        """
        Agente que siempre elige una acción aleatoriamente con probabilidad uniforme.
        """
        self.action_space_size = action_space_size


    def get_action(self, obs: tuple[int, int, bool]) -> int:
        """
        Selecciona una acción aleatoriamente con probabilidad uniforme.
        """
        return random.randint(0, self.action_space_size - 1)

    #obs_past, action, reward, terminated, truncated, info, obs_current
    def update(
        self,
        obs_past: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        truncated: bool,
        info: dict,
        obs_current: tuple[int, int, bool],
    ):
        """ Do nothing"""
        pass


# b.2
agent = BlackjackAgentRandom(env.action_space.n)


### Definir ciclo de entrenamiento básico

**c**) El aprendizaje se realiza en el ciclo de entrenamiento. Aunque dependerá del algoritmo implementado, el esquema tipo puede ser:

- Definir el entorno con el que se trabajará y crear una instancia.
- Implementar el agente, incluida la política.
- Interactuar con el entorno, tomando acciones y recibiendo recompensas.
- Actualizar la política en cada paso o al final del episodio.
- Repetir mientras no se cumpla algún criterio de parada.

En la siguiente casilla se implementa un ciclo básico de entrenamiento. Un episodio consiste en una secuencia de interacción-acción-recompensa. Incluya el código necesario para:

**c.1)** Reiniciar el entorno al comenzar un nuevo episodio. Guarde el estado en una variable llamada `obs_past`.

**c.2)** Pedir una acción al agente, según el estado actual.

**c.3)** Avanzar un paso en el tiempo. Capture las variables `obs_current`, `reward`, `terminated` `truncated` e `info`.

**c.4)** Actualizar la política.

**Sugerencia**: Revise la implementación del ciclo de entrenamiento para comprender el propósito de cada sesión.

In [53]:
from tqdm import tqdm

def training_loop(env, agent, episodes=100, action_meanings={0: 'Stick', 1: 'Hit'}, verbose=False):
    returns = []  # Lista para almacenar los retornos totales de cada episodio

    for eid in tqdm(range(episodes)):
        if verbose:
            print(f'==== Episodio {eid} ====')

        obs_past = env.reset()
        total_reward = 0  # Inicializar el total de recompensas para este episodio

        while True:
            if verbose:
                if len(obs_past) >= 3:
                    print(f'Estado anterior: Puntos={obs_past[0]}  Carta crupier={obs_past[1]}  Aces={obs_past[2]}')
                else:
                    print(f'Estado anterior: Puntos={obs_past[0][0]}  Carta crupier={obs_past[0][1]}  Aces={obs_past[0][2]}')

            action = agent.get_action(obs_past)

            if verbose:
                print(f'Acción: {action_meanings[action]}\n')

            obs_current, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward  # Acumular recompensa

            agent.update(obs_past, action, reward, terminated, truncated, {}, obs_current)

            obs_past = obs_current

            if verbose:
                print(f'Estado actual: Puntos={obs_current[0]}  Carta crupier={obs_current[1]}  Aces={obs_current[2]}')

            if terminated:
                if verbose:
                    print(f'Fin del juego! Recompensa={reward} ')
                    print('Ganaste :)\n\n') if reward > 0 else print('Perdiste :(\n\n') if reward < 0 else print('Empate!\n\n')

                break

        returns.append(total_reward)  # Guardar el retorno total para este episodio

    return returns





### Realizar ciclos de entrenamiento (de prueba)

Ejecute la siguiente casilla para generar $5$ episodios del juego.

In [54]:
training_loop(env, agent, episodes=5, verbose=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 6432.98it/s]

==== Episodio 0 ====
Estado anterior: Puntos=18  Carta crupier=4  Aces=0
Acción: Stick

Estado actual: Puntos=18  Carta crupier=4  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 1 ====
Estado anterior: Puntos=16  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=26  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2 ====
Estado anterior: Puntos=17  Carta crupier=8  Aces=0
Acción: Hit

Estado actual: Puntos=24  Carta crupier=8  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3 ====
Estado anterior: Puntos=20  Carta crupier=4  Aces=0
Acción: Stick

Estado actual: Puntos=20  Carta crupier=4  Aces=0
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 4 ====
Estado anterior: Puntos=17  Carta crupier=5  Aces=1
Acción: Hit

Estado actual: Puntos=20  Carta crupier=5  Aces=1
Estado anterior: Puntos=20  Carta crupier=5  Aces=1
Acción: Hit

Estado actual: Puntos=20  Carta crupier=5  Aces=0
Estado anterior: 

[-1.0, -1.0, -1.0, 1.0, -1.0]

### Añadir un "decorador" al entorno.

**d)** En Gymnasium, los "[decoradores](https://gymnasium.farama.org/api/wrappers/)" permiten añadir funcionalidades adicionales a un entorno, por ejemplo, llevar un registro de los retornos de cada episodio, con el fin de analizar el comportamiento de estos a medida que el agente aprende.

En la siguiente casilla, incluya el código necesario para:

**d.1)** Crear e iniciar un decorador de tipo RecordEpisodeStatistics para el entorno. Utilice una variable llamada `envw`.

**d.2)** Imprimir el tamaño del espacio de acciones y la configuración de un estado. El código necesario ya se provee.

**d.3)** Ejecutar un paso de tiempo en el entorno mediante la función `step`. Capture todos los valores que devuelve esta función en las variables `obs_current`, `reward`, `terminated` `truncated` e `info`. Utilice la función `sample` del `action_espace` del entorno para determinar la acción.

**d.4)** Imprimir el estado actual y la recompensa obtenida. El código necesario ya se provee.

In [59]:
# d.1


episodes = 10000 # número de episodios a generar
envw = gym.wrappers.RecordEpisodeStatistics(env) # envoltura para almacenar estadísticas de los episodios
envw.reset() # reiniciar el entorno

returns = training_loop(envw, agent, episodes=episodes, verbose=True)

# d.2
print(f'Espacio de Acciones: {envw.action_space}')
print(f'Espacio de Observaciones (estados): {envw.observation_space}')

# d.3
action = agent.get_action(obs_current)
obs_current, reward, terminated, truncated, _ = envw.step(action)

# d.4
print(f'Estado actual: Puntos={obs_current[0]}  Carta crupier={obs_current[1]}  Aces={obs_current[2]}')

 23%|█████████████████████▍                                                                         | 2260/10000 [00:00<00:00, 11454.81it/s]

==== Episodio 0 ====
Estado anterior: Puntos=14  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=14  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 1 ====
Estado anterior: Puntos=12  Carta crupier=1  Aces=0
Acción: Hit

Estado actual: Puntos=13  Carta crupier=1  Aces=0
Estado anterior: Puntos=13  Carta crupier=1  Aces=0
Acción: Hit

Estado actual: Puntos=23  Carta crupier=1  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2 ====
Estado anterior: Puntos=16  Carta crupier=9  Aces=1
Acción: Hit

Estado actual: Puntos=14  Carta crupier=9  Aces=0
Estado anterior: Puntos=14  Carta crupier=9  Aces=0
Acción: Stick

Estado actual: Puntos=14  Carta crupier=9  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3 ====
Estado anterior: Puntos=16  Carta crupier=5  Aces=0
Acción: Stick

Estado actual: Puntos=16  Carta crupier=5  Aces=0
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 4 ====
Estado anterior

 46%|███████████████████████████████████████████▊                                                   | 4617/10000 [00:00<00:00, 11580.96it/s]

Estado anterior: Puntos=19  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=29  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2610 ====
Estado anterior: Puntos=8  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=8  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2611 ====
Estado anterior: Puntos=14  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=14  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2612 ====
Estado anterior: Puntos=12  Carta crupier=8  Aces=0
Acción: Stick

Estado actual: Puntos=12  Carta crupier=8  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2613 ====
Estado anterior: Puntos=8  Carta crupier=5  Aces=0
Acción: Stick

Estado actual: Puntos=8  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2614 ====
Estado anterior: Puntos=13  Carta crupier=8  Aces=1
Acción: Hit



 70%|██████████████████████████████████████████████████████████████████▋                            | 7023/10000 [00:00<00:00, 11878.29it/s]

Estado anterior: Puntos=17  Carta crupier=7  Aces=0
Acción: Stick

Estado actual: Puntos=17  Carta crupier=7  Aces=0
Fin del juego! Recompensa=0.0 
Empate!


==== Episodio 5118 ====
Estado anterior: Puntos=11  Carta crupier=1  Aces=0
Acción: Hit

Estado actual: Puntos=18  Carta crupier=1  Aces=0
Estado anterior: Puntos=18  Carta crupier=1  Aces=0
Acción: Stick

Estado actual: Puntos=18  Carta crupier=1  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5119 ====
Estado anterior: Puntos=13  Carta crupier=1  Aces=0
Acción: Stick

Estado actual: Puntos=13  Carta crupier=1  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5120 ====
Estado anterior: Puntos=21  Carta crupier=10  Aces=1
Acción: Hit

Estado actual: Puntos=12  Carta crupier=10  Aces=0
Estado anterior: Puntos=12  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=17  Carta crupier=10  Aces=0
Estado anterior: Puntos=17  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=27  Car

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 11660.60it/s]

Estado actual: Puntos=8  Carta crupier=2  Aces=0
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 8018 ====
Estado anterior: Puntos=13  Carta crupier=6  Aces=0
Acción: Stick

Estado actual: Puntos=13  Carta crupier=6  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 8019 ====
Estado anterior: Puntos=17  Carta crupier=8  Aces=1
Acción: Hit

Estado actual: Puntos=17  Carta crupier=8  Aces=0
Estado anterior: Puntos=17  Carta crupier=8  Aces=0
Acción: Hit

Estado actual: Puntos=23  Carta crupier=8  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 8020 ====
Estado anterior: Puntos=15  Carta crupier=5  Aces=0
Acción: Hit

Estado actual: Puntos=25  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 8021 ====
Estado anterior: Puntos=21  Carta crupier=10  Aces=1
Acción: Stick

Estado actual: Puntos=21  Carta crupier=10  Aces=1
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 8022 ====
Estado anterior: Puntos=

### Integrar entorno, agente y ciclo de entrenamiento.


**e)** Finalmente, integramos el entorno, el agente y el ciclo de entrenamiento para realizar el aprendizaje.

En la siguiente casilla, incluya el código necesario para:

**e.1)** Llamar a la función que implementa el ciclo de entrenamiento para generar $10K$ episodios utilizando `envw`. El código necesario ya se provee.

**e.2)** Graficar mediante un gráfico de dispersión el promedio de episodios ganados hasta el momento. Notar que el retorno es $+1$ en estos. El código necesario ya se provee.

In [61]:
# e.1
training_loop(envw, agent, episodes=10000, verbose=True)
print(f'Promedio de episodios ganados: {returns}')

# e.2
y = np.array(envw.return_queue).flatten()
x = np.arange(0, y.shape[0])
fig = visualize_trace(x, y)
fig.show()

 21%|████████████████████▏                                                                          | 2119/10000 [00:00<00:00, 10879.33it/s]

==== Episodio 0 ====
Estado anterior: Puntos=16  Carta crupier=7  Aces=0
Acción: Hit

Estado actual: Puntos=24  Carta crupier=7  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 1 ====
Estado anterior: Puntos=15  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=15  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 2 ====
Estado anterior: Puntos=14  Carta crupier=5  Aces=0
Acción: Stick

Estado actual: Puntos=14  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3 ====
Estado anterior: Puntos=10  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=10  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 4 ====
Estado anterior: Puntos=15  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=15  Carta crupier=10  Aces=0
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 5 ====
Estado anterior: Puntos=13  Carta crupier=1  Aces=1
Acció

 59%|███████████████████████████████████████████████████████▌                                       | 5852/10000 [00:00<00:00, 12220.41it/s]

Estado actual: Puntos=4  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3012 ====
Estado anterior: Puntos=14  Carta crupier=1  Aces=0
Acción: Stick

Estado actual: Puntos=14  Carta crupier=1  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3013 ====
Estado anterior: Puntos=14  Carta crupier=5  Aces=0
Acción: Hit

Estado actual: Puntos=23  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3014 ====
Estado anterior: Puntos=15  Carta crupier=4  Aces=1
Acción: Stick

Estado actual: Puntos=15  Carta crupier=4  Aces=1
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3015 ====
Estado anterior: Puntos=11  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=19  Carta crupier=10  Aces=0
Estado anterior: Puntos=19  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=24  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 3016 ====
Estado anterior: 

 83%|███████████████████████████████████████████████████████████████████████████████▏               | 8335/10000 [00:00<00:00, 12141.76it/s]

==== Episodio 5852 ====
Estado anterior: Puntos=20  Carta crupier=5  Aces=0
Acción: Hit

Estado actual: Puntos=30  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5853 ====
Estado anterior: Puntos=12  Carta crupier=5  Aces=0
Acción: Stick

Estado actual: Puntos=12  Carta crupier=5  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5854 ====
Estado anterior: Puntos=17  Carta crupier=3  Aces=0
Acción: Hit

Estado actual: Puntos=22  Carta crupier=3  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5855 ====
Estado anterior: Puntos=15  Carta crupier=1  Aces=0
Acción: Stick

Estado actual: Puntos=15  Carta crupier=1  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 5856 ====
Estado anterior: Puntos=10  Carta crupier=6  Aces=0
Acción: Hit

Estado actual: Puntos=20  Carta crupier=6  Aces=0
Estado anterior: Puntos=20  Carta crupier=6  Aces=0
Acción: Hit

Estado actual: Puntos=30  Carta crupier=6  Aces=0
Fi

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 12039.77it/s]

==== Episodio 9735 ====
Estado anterior: Puntos=17  Carta crupier=10  Aces=0
Acción: Hit

Estado actual: Puntos=24  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 9736 ====
Estado anterior: Puntos=16  Carta crupier=6  Aces=0
Acción: Hit

Estado actual: Puntos=21  Carta crupier=6  Aces=0
Estado anterior: Puntos=21  Carta crupier=6  Aces=0
Acción: Stick

Estado actual: Puntos=21  Carta crupier=6  Aces=0
Fin del juego! Recompensa=1.0 
Ganaste :)


==== Episodio 9737 ====
Estado anterior: Puntos=15  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=15  Carta crupier=10  Aces=0
Fin del juego! Recompensa=-1.0 
Perdiste :(


==== Episodio 9738 ====
Estado anterior: Puntos=19  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=19  Carta crupier=10  Aces=0
Fin del juego! Recompensa=0.0 
Empate!


==== Episodio 9739 ====
Estado anterior: Puntos=13  Carta crupier=10  Aces=0
Acción: Stick

Estado actual: Puntos=13  Carta crupier=10  Aces